# New York City Taxi Fare Prediction

## Reading data and first exploration
We will need various libraries in this notebook.

In [78]:
# load some default Python modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-whitegrid')

% matplotlib inline

In [10]:
# read data in pandas dataframe
df_train =  pd.read_csv('NYC_Taxi/train.csv', nrows = 2000000, parse_dates=["pickup_datetime"])


In [14]:
import os

In [16]:
os.makedirs('tmp')
df_train.to_feather('tmp/df_train-raw')

LOading the data takes quite some time therefore we have saved the df in feather format for faster loading.

In [ ]:
import feather
df_train = feather.read_dataframe('tmp/df_train-raw')

In [ ]:
df_train.head()

In [ ]:
# check datatypes
df_train.dtypes

In [ ]:
df_train.describe()

There are some negative fare values which is not realistic

In [ ]:
len(df_train[df_train.fare_amount<0])

we can drop the 77 entries

In [ ]:
df_train = df_train[df_train.fare_amount>=0]

In [ ]:
len(df_train[df_train.fare_amount<0])

## Missing data

Missing data will affect our models and analysis It is important to fix this,

In [ ]:
df_train.isnull().sum()

there are less than 30 null entries we can drop them

In [ ]:
#df_train = df_train.dropna(inplace=True,  axis = 'rows')

In [ ]:
df_train.head()

In [ ]:
df_train.to_feather('tmp/df_train-raw')